In [9]:
import pandas as pd
import yfinance as yf
from datetime import datetime

start_date = datetime.now() - pd.DateOffset(months=12)
end_date = datetime.now()

tickers = ['AAPL', 'MSFT', 'NFLX', 'GOOG']

df_list = []

for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    df_list.append(data)

df = pd.concat(df_list, keys=tickers, names=['Ticker', 'Date'])
print(df.head())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

                         Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
AAPL   2023-11-21  191.410004  191.520004  189.740005  190.639999  189.710587   
       2023-11-22  191.490005  192.929993  190.830002  191.309998  190.377335   
       2023-11-24  190.869995  190.899994  189.250000  189.970001  189.043854   
       2023-11-27  189.919998  190.669998  188.899994  189.789993  188.864731   
       2023-11-28  189.779999  191.080002  189.399994  190.399994  189.471756   

                     Volume  
Ticker Date                  
AAPL   2023-11-21  38134500  
       2023-11-22  39617700  
       2023-11-24  24048300  
       2023-11-27  40552600  
       2023-11-28  38415400  


In [10]:
df = df.reset_index()
print(df.head())

  Ticker       Date        Open        High         Low       Close  \
0   AAPL 2023-11-21  191.410004  191.520004  189.740005  190.639999   
1   AAPL 2023-11-22  191.490005  192.929993  190.830002  191.309998   
2   AAPL 2023-11-24  190.869995  190.899994  189.250000  189.970001   
3   AAPL 2023-11-27  189.919998  190.669998  188.899994  189.789993   
4   AAPL 2023-11-28  189.779999  191.080002  189.399994  190.399994   

    Adj Close    Volume  
0  189.710587  38134500  
1  190.377335  39617700  
2  189.043854  24048300  
3  188.864731  40552600  
4  189.471756  38415400  


In [11]:
import plotly.express as px
fig = px.line(df, x='Date', 
              y='Close', 
              color='Ticker', 
              title="Stock Market Performance for the Last 12 Months")
fig.show()

In [12]:
fig = px.area(df, x='Date', y='Close', color='Ticker',
              facet_col='Ticker',
              labels={'Date':'Date', 'Close':'Closing Price', 'Ticker':'Company'},
              title='Stock Prices for Apple, Microsoft, Netflix, and Google')
fig.show()

In [13]:
df['MA10'] = df.groupby('Ticker')['Close'].rolling(window=10).mean().reset_index(0, drop=True)
df['MA20'] = df.groupby('Ticker')['Close'].rolling(window=20).mean().reset_index(0, drop=True)

for ticker, group in df.groupby('Ticker'):
    print(f'Moving Averages for {ticker}')
    print(group[['MA10', 'MA20']])

Moving Averages for AAPL
           MA10        MA20
0           NaN         NaN
1           NaN         NaN
2           NaN         NaN
3           NaN         NaN
4           NaN         NaN
..          ...         ...
246  224.501999  228.720999
247  224.732999  228.524500
248  224.941998  228.024500
249  225.542999  227.601500
250  226.025999  227.222500

[251 rows x 2 columns]
Moving Averages for GOOG
            MA10        MA20
753          NaN         NaN
754          NaN         NaN
755          NaN         NaN
756          NaN         NaN
757          NaN         NaN
...          ...         ...
999   177.368001  172.374001
1000  177.834001  173.016001
1001  177.958002  173.458001
1002  178.570003  174.008001
1003  179.387003  174.646001

[251 rows x 2 columns]
Moving Averages for MSFT
           MA10        MA20
251         NaN         NaN
252         NaN         NaN
253         NaN         NaN
254         NaN         NaN
255         NaN         NaN
..          ...         .

In [14]:
for ticker, group in df.groupby('Ticker'):
    fig = px.line(group, x='Date', y=['Close', 'MA10', 'MA20'], 
                  title=f"{ticker} Moving Averages")
    fig.show()

In [15]:
df['Volatility'] = df.groupby('Ticker')['Close'].pct_change().rolling(window=10).std().reset_index(0, drop=True)
fig = px.line(df, x='Date', y='Volatility', 
              color='Ticker', 
              title='Volatility of All Companies')
fig.show()

In [16]:
# create a DataFrame with the stock prices of Apple and Microsoft
apple = df.loc[df['Ticker'] == 'AAPL', ['Date', 'Close']].rename(columns={'Close': 'AAPL'})
microsoft = df.loc[df['Ticker'] == 'MSFT', ['Date', 'Close']].rename(columns={'Close': 'MSFT'})
df_corr = pd.merge(apple, microsoft, on='Date')

# create a scatter plot to visualize the correlation
fig = px.scatter(df_corr, x='AAPL', y='MSFT', 
                 trendline='ols', 
                 title='Correlation between Apple and Microsoft')
fig.show()